In [1]:
import sys
sys.path.append('..')

from deployment_simulator import DeploymentSimulator
import config
import pandas as pd

print("Imports successful")

Configuration loaded successfully!
Project root: C:\Users\hp\Desktop\gaf\Ghana-Armed-Forces-Personnel-Deployment-and-Attrition-Risk-Modeling
Random seed: 42
Target sample size: 1000 personnel
Imports successful


## Initialize Simulator

In [2]:
simulator = DeploymentSimulator(
    data_path=config.PROCESSED_DATA_DIR / 'features_engineered.csv',
    attrition_model_path=config.MODELS_DIR / 'attrition_classifier.pkl',
    readiness_model_path=config.MODELS_DIR / 'readiness_regressor.pkl',
    readiness_scaler_path=config.MODELS_DIR / 'readiness_scaler.pkl'
)

print("Simulator initialized")

Loaded 1000 personnel records
Models loaded
Attrition features: 100
Readiness features: 98
Simulator initialized


## Scenario 1: Standard Deployment

Deploy 50 personnel for 6 months with minimum readiness of 80.

In [3]:
results_standard = simulator.simulate_deployment(
    n_personnel=50,
    duration_months=6,
    min_readiness=80,
    allow_high_risk=False,
    strategy='balanced'
)


--------------------------------------------------------------------------------
DEPLOYMENT SIMULATION
--------------------------------------------------------------------------------
Personnel needed: 50
Duration: 6 months
Min readiness: 80
Strategy: balanced

After readiness filter (>=80): 137 eligible
After risk filter (no HIGH_RISK): 128 eligible

--------------------------------------------------------------------------------
DEPLOYMENT OUTCOMES
--------------------------------------------------------------------------------
Selected: 50 personnel
Average readiness: 85.3
Risk distribution: {'LOW_RISK': 37, 'MEDIUM_RISK': 13}
Expected attrition: 0 personnel (1.8%)
Replacement needed: 0

--------------------------------------------------------------------------------
ALTERNATIVE SCENARIOS
--------------------------------------------------------------------------------

1. Lower readiness to 75
   eligible_pool: 293
   avg_readiness: 79.9
   trade_off: 293 total personnel available


## Scenario 2: Specialized Mission

Deploy specific MOS with higher readiness requirements.

In [4]:
results_specialized = simulator.simulate_deployment(
    n_personnel=30,
    duration_months=9,
    min_readiness=85,
    mos_requirements=['Infantry', 'Medical', 'Special Forces'],
    allow_high_risk=False,
    strategy='readiness'
)


--------------------------------------------------------------------------------
DEPLOYMENT SIMULATION
--------------------------------------------------------------------------------
Personnel needed: 30
Duration: 9 months
Min readiness: 85
Strategy: readiness

After readiness filter (>=85): 25 eligible
After risk filter (no HIGH_RISK): 22 eligible
After MOS filter ['Infantry', 'Medical', 'Special Forces']: 3 eligible

Consider lowering requirements or see alternative scenarios below.

--------------------------------------------------------------------------------
DEPLOYMENT OUTCOMES
--------------------------------------------------------------------------------
Selected: 3 personnel
Average readiness: 87.9
Risk distribution: {'MEDIUM_RISK': 2, 'LOW_RISK': 1}
Expected attrition: 0 personnel (9.2%)
Replacement needed: 0

--------------------------------------------------------------------------------
ALTERNATIVE SCENARIOS
-------------------------------------------------------------

## Scenario 3: Emergency Deployment

Lower requirements to meet urgent needs.

In [5]:
results_emergency = simulator.simulate_deployment(
    n_personnel=100,
    duration_months=3,
    min_readiness=70,
    allow_high_risk=True,
    strategy='balanced'
)


--------------------------------------------------------------------------------
DEPLOYMENT SIMULATION
--------------------------------------------------------------------------------
Personnel needed: 100
Duration: 3 months
Min readiness: 70
Strategy: balanced

After readiness filter (>=70): 549 eligible

--------------------------------------------------------------------------------
DEPLOYMENT OUTCOMES
--------------------------------------------------------------------------------
Selected: 100 personnel
Average readiness: 83.8
Risk distribution: {'LOW_RISK': 68, 'MEDIUM_RISK': 31, 'HIGH_RISK': 1}
Expected attrition: 1 personnel (1.1%)
Replacement needed: 1

--------------------------------------------------------------------------------
ALTERNATIVE SCENARIOS
--------------------------------------------------------------------------------


## Compare Selection Strategies

Which strategy optimizes for different goals?

In [6]:
strategy_results, comparison_df = simulator.compare_strategies(
    n_personnel=50,
    duration_months=6,
    min_readiness=80
)

print("\nStrategy Recommendations:")
print("- Use 'readiness' for high-stakes missions")
print("- Use 'low_risk' for long-term deployments")
print("- Use 'balanced' for standard operations")


--------------------------------------------------------------------------------
STRATEGY COMPARISON
--------------------------------------------------------------------------------

--------------------------------------------------------------------------------
DEPLOYMENT SIMULATION
--------------------------------------------------------------------------------
Personnel needed: 50
Duration: 6 months
Min readiness: 80
Strategy: readiness

After readiness filter (>=80): 137 eligible
After risk filter (no HIGH_RISK): 128 eligible

--------------------------------------------------------------------------------
DEPLOYMENT OUTCOMES
--------------------------------------------------------------------------------
Selected: 50 personnel
Average readiness: 85.5
Risk distribution: {'LOW_RISK': 31, 'MEDIUM_RISK': 19}
Expected attrition: 1 personnel (2.8%)
Replacement needed: 1

--------------------------------------------------------------------------------
ALTERNATIVE SCENARIOS
------------

## View Selected Personnel

Examine who would be deployed in the standard scenario.

In [7]:
selected_df = pd.DataFrame(results_standard['selected_personnel'])
print(f"\nDeployed Personnel (n={len(selected_df)}):")
print(selected_df.head(10))

print("\nSummary Statistics:")
print(selected_df[['age', 'years_of_service', 'predicted_readiness_score']].describe())


Deployed Personnel (n=50):
   age     rank             MOS  years_of_service  predicted_readiness_score  \
0   34      NCO        Infantry                13                  90.420669   
1   27      NCO          Signal                11                  90.012839   
2   30  Officer     Maintenance                22                  90.040905   
3   36  Officer  Administration                10                  90.030928   
4   35      NCO  Administration                 6                  89.788888   
5   30  Officer        Engineer                28                  88.598611   
6   29      NCO        Engineer                17                  87.620251   
7   35      NCO           Armor                 6                  87.498279   
8   31      NCO  Special Forces                 9                  87.247437   
9   33  Officer    Intelligence                27                  87.262129   

  predicted_attrition_risk  
0                 LOW_RISK  
1                 LOW_RISK  
2   

##  Export Deployment List

Save selected personnel for commanders.

In [8]:
selected_df.to_csv(config.REPORTS_DIR / 'deployment_list_scenario1.csv', index=False)
print("Deployment list saved")

Deployment list saved
